### Step 1: Setup imports & libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import os
import cv2
import pandas as pd
from classes.GradCAM import GradCAM

### Step 2: Configure Variables

In [ ]:
model_name = "new_model" # The name of the model to save.
batch_size = 32 # Number of samples processed before the model's weights are updated.
img_height = 500 # Height of the input images to be resized to before being fed into the model.
img_width = 500 # Width of the input images to be resized to before being fed into the model.
epochs = 10 # Number of complete passes through the training dataset.
training_dir = "train/" # Path to the training dataset.
testing_dir = "test/" # Path to the testing dataset.
show_heatmap = True # Whether to show the heatmap of the Grad-CAM.
seed=123 # Seed for reproducibility.

### Step 3: Set Directory for Training Data

In [ ]:
# Count total number of images in the training data
data_dir = pathlib.Path(training_dir).with_suffix("")
total_image_count = len(list(data_dir.glob('*/*.jpg')))

class_names = [] # Array to store class names
class_counts = []  # Array to store counts
total_image_count = 0  # Initialise total count

# Count number of training images in each subfolder
for folder in data_dir.iterdir():
    if folder.is_dir():
        image_count = len(list(folder.glob('*.jpg')))
        class_counts.append(image_count)  # Save count in the array
        class_names.append(folder.name)
        total_image_count += image_count  # Update total count
        print(f"Number of {folder.name} images: {image_count}")

print(f"Total number of images: {total_image_count}\n")

### Step 4: Generate Class Distirbution Graph
Used to check for data imbalances

In [ ]:
class_counts = pd.Series(class_counts)

plt.figure(figsize=(10, 10))
class_counts.plot.pie(labels=class_names, autopct='%1.1f%%', startangle=90, counterclock=False)
plt.ylabel('')  
plt.title('Class Distribution')
plt.savefig("class_distribution.png")
plt.show()

### Step 5: Load & Partition Data

In [ ]:
# Training Data
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

# Validation Data
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=seed,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

class_names = train_ds.class_names
print(f"Class Names: {class_names}")

### Step 6: Visualise Training Data Samples With Class Labels

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

### Step 7. Configure the Dataset for Optimal Performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Step 8. Normalise Pixel Values of Dataset Images

In [ ]:
normalization_layer = layers.Rescaling(1./255) # Normalisation layer to scale pixel values from [0, 255] to [0, 1]

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Apply the normalisation layer to the training data

# Get a batch of normalised images
image_batch, labels_batch = next(iter(normalized_ds)) 
first_image = image_batch[0]

print(np.min(first_image), np.max(first_image)) # Pixel values are now in `[0,1]`.

### Step 9. Build CNN Model Architecture
In this step, we construct a CNN model using the `Sequential` API. The model architecture consists of several convolutional layers, pooling layers, and fully connected (dense) layers, designed to classify images into multiple categories.

In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

### Step 10. Compile CNN Model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

### Step 11. Train the Model

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

### Step 12. Visualising Training and Validation Performance

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.savefig(f"{model_name}_accuracy_loss.png", bbox_inches = 'tight')

plt.show()

### Step 13. Test the Model


In [ ]:
y_true_all = []
y_pred_all = []

for images, labels in val_ds:
    predictions = model(images)
    y_pred = tf.argmax(predictions, axis=1) 
    y_true = labels

    y_true_all.extend(y_true.numpy())
    y_pred_all.extend(y_pred.numpy())

### Step 9. Evaluate the Model
Calculate & visualise the model accuracy, confusion matrix & classification report

In [ ]:
# Convert to numpy arrays
y_true_all = np.array(y_true_all)
y_pred_all = np.array(y_pred_all)

accuracy = accuracy_score(y_true_all, y_pred_all)
print(f"Accuracy (CNN): {accuracy * 100:.2f}%")


# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_true_all, y_pred_all)

print("Classification Report:\n", classification_report(y_true_all, y_pred_all))

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.title('CNN Confusion Matrix')
plt.savefig(f"{model_name}_confusion_matrix.png", bbox_inches = 'tight')
plt.show()

### Step 15. Predict Classes of Unseen Test Images

In [ ]:
processed_images = []
titles = []

# Loop over all files in the test folder and its subfolders
for root, dirs, files in os.walk(testing_dir):
    for test_image in files:
        # Ensure it's a valid image file
        if test_image.endswith(('.jpg', '.jpeg', '.png')):
            # Construct the full path to the image
            image_path = os.path.join(root, test_image)

            orig = cv2.imread(image_path)
            resized = cv2.resize(orig, (img_height, img_width))

            # Load and preprocess the image
            img = tf.keras.utils.load_img(image_path, target_size=(img_height, img_width))
            img_array = tf.keras.utils.img_to_array(img)
            img_array = tf.expand_dims(img_array, 0)  # Create a batch

            # Make predictions using the model
            predictions = model.predict(img_array)
            score = tf.nn.softmax(predictions[0])
            i = np.argmax(predictions[0])

            if show_heatmap:
                # Generate the heatmap using Grad-CAM
                cam = GradCAM(model, i, 'max_pooling2d_2')
                heatmap = cam.compute_heatmap(img_array)

                heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
                (heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)

                # Plot the original and heatmap overlay in a single figure
                plt.figure(figsize=(15, 5))

                # Original Image
                plt.subplot(1, 3, 1)
                plt.imshow(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))
                plt.title('Original Image')
                plt.axis('off')

                # Heatmap Overlay
                plt.subplot(1, 3, 2)
                plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
                plt.title('Heatmap Overlay')
                plt.axis('off')

                # Show the combined plot
                plt.tight_layout()
                plt.savefig(f"{model_name}_{test_image}_heatmap.png", bbox_inches = 'tight')
                plt.show()

            # Add the image and title to the lists
            processed_images.append(img)
            titles.append(f"{test_image} ({class_names[np.argmax(score)]} - {100 * np.max(score):.2f}%)")

### Step 16. Visualise Predictions

In [ ]:
num_images = len(processed_images)
cols = 4  # Number of columns in the grid
rows = (num_images // cols) + 1 if num_images % cols != 0 else num_images // cols

fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))

for i, ax in enumerate(axes.flat):
    if i < num_images:
        ax.imshow(processed_images[i])
        ax.set_title(titles[i])
        ax.axis('off')
    else:
        ax.axis('off')  # Hide any unused subplots

plt.tight_layout()
plt.savefig(f"{model_name}_predictions.png", bbox_inches = 'tight')
plt.show()

### Step 17. Save the model

In [ ]:
model.save(f"{model_name}.keras")

### Step 18. Convert & Save Model as TF Lite

In [ ]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open(f"{model_name}.tflite", 'wb') as f:
  f.write(tflite_model)